#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

# 導入必要的函數

In [1]:
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

Using TensorFlow backend.


# 資料準備

In [2]:
#取得Keras Dataset
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [3]:
#確認 CIFAR10 Dataset 資料維度
print("train data:",'images:',x_img_train.shape,
      " labels:",y_label_train.shape) 
print("test  data:",'images:',x_img_test.shape ,
      " labels:",y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
#資料正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0

In [5]:
#針對Label 做 ONE HOT ENCODE
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
y_label_test_OneHot.shape

(10000, 10)

# 建立模型

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [7]:
model = Sequential()

In [8]:
#卷積層1

In [9]:
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32, 32,3), 
                 activation='relu', 
                 padding='same'))

In [10]:
model.add(Dropout(rate=0.25))

In [11]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [12]:
#卷積層2與池化層2

In [13]:
model.add(Conv2D(filters=64, kernel_size=(3, 3), 
                 activation='relu', padding='same'))

In [14]:
model.add(Dropout(0.25))

In [15]:
model.add(MaxPooling2D(pool_size=(2, 2)))

In [16]:
#建立神經網路(平坦層、隱藏層、輸出層)

In [17]:
model.add(Flatten())
model.add(Dropout(rate=0.25))

In [18]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(rate=0.25))

In [19]:
model.add(Dense(10, activation='softmax'))

In [20]:
#檢查model 的STACK
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

# 載入之前訓練的模型

In [21]:
try:
    model.load_weights("SaveModel/cifarCnnModel.h5")
    print("載入模型成功!繼續訓練模型")
except :    
    print("載入模型失敗!開始訓練一個新模型")

載入模型失敗!開始訓練一個新模型


# 訓練模型

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.save_weights("./071HW.h5")

'''
作業:
請分別選用 "MSE", "binary _crossentropy"
查看Train/test accurancy and loss rate
'''

'\n作業:\n請分別選用 "MSE", "binary _crossentropy"\n查看Train/test accurancy and loss rate\n'

In [23]:
#模型訓練, "Train_History" 把訓練過程所得到的數值存起來
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)         

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 5s 122us/step - loss: 2.1650 - acc: 0.1987 - val_loss: 2.0515 - val_acc: 0.2954
Epoch 2/12
37500/37500 [==============================] - 3s 89us/step - loss: 1.9549 - acc: 0.2961 - val_loss: 1.9353 - val_acc: 0.3459
Epoch 3/12
37500/37500 [==============================] - 3s 84us/step - loss: 1.8338 - acc: 0.3471 - val_loss: 1.8413 - val_acc: 0.3827
Epoch 4/12
37500/37500 [==============================] - 3s 79us/step - loss: 1.7301 - acc: 0.3818 - val_loss: 1.7483 - val_acc: 0.4177
Epoch 5/12
37500/37500 [==============================] - 3s 80us/step - loss: 1.6493 - acc: 0.4102 - val_loss: 1.6799 - val_acc: 0.4405
Epoch 6/12
37500/37500 [==============================] - 3s 81us/step - loss: 1.5852 - acc: 0.4309 - val_loss: 1.6497 - val_acc: 0.4515
Epoch 7/12
37500/37500 [==============================] - 4s 94us/step - loss: 1.5309 - acc: 0.4541 - val_loss: 1.5881 - val_ac

In [24]:

scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print('accuracy=',scores[1])

10000/10000 [==============================] - 1s 103us/step
accuracy= 0.5468


In [25]:
prediction = model.predict_classes(x_img_test_normalize)
import pandas as pd
pd.crosstab(y_label_test.reshape(-1), prediction, rownames=['label'], colnames=['predict'])

predict,0,1,2,3,4,5,6,7,8,9
label,,,,,,,,,,
0,597,20,96,22,29,12,29,26,130,39
1,45,579,23,15,9,11,46,31,58,183
2,62,6,401,52,188,86,123,53,19,10
3,13,4,104,302,104,209,178,56,6,24
4,24,4,108,47,531,54,135,80,13,4
5,11,2,105,140,80,472,88,84,13,5
6,2,5,69,49,97,20,725,19,4,10
7,13,2,55,47,107,94,54,604,6,18
8,117,48,41,16,18,17,19,20,655,49


In [26]:
from sklearn.metrics import accuracy_score
scores = accuracy_score(y_label_test,prediction)
scores

0.5468

In [27]:
model.load_weights("./071HW.h5")
model.compile(loss='MSE', optimizer='sgd', metrics=['accuracy'])
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)   

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 4s 103us/step - loss: 0.0905 - acc: 0.1057 - val_loss: 0.0898 - val_acc: 0.1326
Epoch 2/12
37500/37500 [==============================] - 3s 87us/step - loss: 0.0901 - acc: 0.1105 - val_loss: 0.0897 - val_acc: 0.1398
Epoch 3/12
37500/37500 [==============================] - 4s 95us/step - loss: 0.0899 - acc: 0.1189 - val_loss: 0.0897 - val_acc: 0.1412
Epoch 4/12
37500/37500 [==============================] - 3s 92us/step - loss: 0.0898 - acc: 0.1220 - val_loss: 0.0896 - val_acc: 0.1482
Epoch 5/12
37500/37500 [==============================] - 4s 94us/step - loss: 0.0897 - acc: 0.1271 - val_loss: 0.0895 - val_acc: 0.1552
Epoch 6/12
37500/37500 [==============================] - 4s 94us/step - loss: 0.0896 - acc: 0.1339 - val_loss: 0.0895 - val_acc: 0.1666
Epoch 7/12
37500/37500 [==============================] - 4s 94us/step - loss: 0.0895 - acc: 0.1366 - val_loss: 0.0894 - val_ac

In [28]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print('accuracy=',scores[1])

10000/10000 [==============================] - 1s 88us/step
accuracy= 0.2008


In [29]:
prediction = model.predict_classes(x_img_test_normalize)
import pandas as pd
pd.crosstab(y_label_test.reshape(-1), prediction, rownames=['label'], colnames=['predict'])

predict,0,1,2,3,4,5,6,7,8,9
label,,,,,,,,,,
0,340,0,1,1,2,15,4,5,474,158
1,112,1,0,2,18,82,24,5,449,307
2,169,2,0,9,22,147,63,29,355,204
3,152,0,0,24,25,241,55,24,227,252
4,93,1,0,12,51,196,106,27,271,243
5,112,1,2,11,33,270,62,23,284,202
6,88,0,0,8,35,264,105,19,191,290
7,86,1,1,7,25,153,43,19,250,415
8,125,1,0,2,3,34,2,2,646,185


In [30]:
from sklearn.metrics import accuracy_score
scores = accuracy_score(y_label_test,prediction)
scores

0.2008

In [31]:
model.load_weights("./071HW.h5")
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
train_history=model.fit(x_img_train_normalize, y_label_train_OneHot,
                        validation_split=0.25,
                        epochs=12, batch_size=128, verbose=1)   

Train on 37500 samples, validate on 12500 samples
Epoch 1/12
37500/37500 [==============================] - 4s 112us/step - loss: 0.3249 - acc: 0.9000 - val_loss: 0.3224 - val_acc: 0.9000
Epoch 2/12
37500/37500 [==============================] - 4s 99us/step - loss: 0.3215 - acc: 0.9000 - val_loss: 0.3203 - val_acc: 0.9000
Epoch 3/12
37500/37500 [==============================] - 4s 103us/step - loss: 0.3181 - acc: 0.9000 - val_loss: 0.3176 - val_acc: 0.9000
Epoch 4/12
37500/37500 [==============================] - 4s 94us/step - loss: 0.3140 - acc: 0.9000 - val_loss: 0.3139 - val_acc: 0.9000
Epoch 5/12
37500/37500 [==============================] - 3s 90us/step - loss: 0.3088 - acc: 0.9000 - val_loss: 0.3093 - val_acc: 0.9000
Epoch 6/12
37500/37500 [==============================] - 4s 102us/step - loss: 0.3034 - acc: 0.9000 - val_loss: 0.3047 - val_acc: 0.9000
Epoch 7/12
37500/37500 [==============================] - 4s 102us/step - loss: 0.2984 - acc: 0.9002 - val_loss: 0.3007 - val

In [32]:
scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot)
print('accuracy=',scores[1])

10000/10000 [==============================] - 1s 96us/step
accuracy= 0.9002099769592286


In [33]:
prediction = model.predict_classes(x_img_test_normalize)
import pandas as pd
pd.crosstab(y_label_test.reshape(-1), prediction, rownames=['label'], colnames=['predict'])

predict,0,1,2,3,4,5,6,7,8,9
label,,,,,,,,,,
0,450,49,24,20,10,23,34,47,256,87
1,73,305,13,19,25,78,115,41,152,179
2,116,43,89,38,75,119,311,95,68,46
3,55,36,55,101,52,256,230,101,32,82
4,46,15,40,42,167,93,412,94,41,50
5,31,31,59,59,46,373,205,104,52,40
6,23,18,43,44,43,99,598,83,8,41
7,29,60,13,39,105,117,143,287,45,162
8,112,58,7,15,3,67,11,29,573,125


In [34]:
from sklearn.metrics import accuracy_score
scores = accuracy_score(y_label_test,prediction)
scores

0.3395